In [1]:
import re
from collections import Counter
import numpy as np
import pandas as pd

## Data Preprocessing

In [2]:
from nltk.tokenize import word_tokenize

def process_data(file_name):
    """
    Input: 
        A file_name which is found by name shakespeare.txt. We just have to read it in. 
    Output: 
        words: a list containing all the words in the corpus in lower case. 
    """
    words = [] 

    with open(file_name, 'r') as file:
        file_read = file.read()
        pre_process_file = re.findall(r"\w+", file_read)
        word_list = pre_process_file
        for word in word_list:
            words.append(word.lower())
    
    return words

In [3]:
word_list = process_data('shakespeare.txt')
vocab = set(word_list)  # this will be our new vocabulary

In [4]:
vocab

{'intellectual',
 'boundless',
 'dignity',
 'performed',
 'hart',
 'proofread',
 'daily',
 'takes',
 'unbred',
 'palate',
 'reign',
 'perspective',
 'all',
 'deceased',
 'walk',
 'damask',
 'habitation',
 'readable',
 'history',
 'begins',
 'feasts',
 'ward',
 'forth',
 'string',
 'night',
 'ow',
 'enjoyer',
 'rich',
 'xxviii',
 'by',
 'prone',
 'sland',
 'autumn',
 'crow',
 'party',
 'heard',
 'lov',
 'acknowledge',
 'cause',
 'suspect',
 'cliv',
 'thanks',
 'forward',
 'salve',
 'windy',
 'carcanet',
 'wardrobe',
 'warranty',
 'jealous',
 'exceed',
 'heretic',
 'monarch',
 'costs',
 'voice',
 'grown',
 'irs',
 'nymphs',
 'hadst',
 'enrich',
 'thralled',
 'leases',
 'profit',
 'pardon',
 'aright',
 'queen',
 'cviii',
 'pleasures',
 'serving',
 'forsworn',
 'pages',
 'importune',
 'runn',
 'wrong',
 'churl',
 'bore',
 'anticipate',
 'impiety',
 'doubting',
 'email',
 'fingers',
 'horse',
 'annoy',
 'put',
 'kiss',
 'xc',
 'well',
 'providing',
 'yourself',
 'disclaimers',
 'feeding',
 

## Word Count

In [5]:
def get_count(word_list):
    '''
    Input:
        word_list: a set of words representing the corpus. 
    Output:
        word_count_dict: The wordcount dictionary where key is the word and value is its frequency.
    '''
    
    word_count_dict = {}  # create word_count dictionary
    word_count_dict = Counter(word_list)
            
    return word_count_dict

In [6]:
word_count_dict = get_count(word_list)
print(f"There are {len(word_count_dict)} key values pairs")
print(f"The count for the word 'thee' is {word_count_dict.get('thee',0)}")

There are 3668 key values pairs
The count for the word 'thee' is 162


## Computing the Word Probability

In [7]:
def get_probs(word_count_dict):
    '''
    Input:
        word_count_dict: The wordcount dictionary where key is the word and value is its frequency.
    Output:
        probs: A dictionary where keys are the words and the values are the probability that a word will occur. 
    '''
    probs = {}  # initalize the probability dictionary
    total_words = sum(word_count_dict.values()) # variable M

    for word, word_count in word_count_dict.items():
        word_prob = word_count/total_words
        probs[word] = word_prob
    
    return probs

In [8]:
print(word_count_dict['thee'])
probs = get_probs(word_count_dict)
print(f"Length of probs is {len(probs)}")
print(f"P('thee') is {probs['thee']:.4f}")

162
Length of probs is 3668
P('thee') is 0.0075


## Performing String Manipulations

### Delete letter

In [9]:
def delete_letter(word, verbose=False):
    '''
    Input:
        word: the string/word for which we will generate all possible words 
                in the vocabulary which have 1 missing character
    Output:
        delete_list: a list of all possible strings obtained by deleting 1 character from word
    '''
    
    delete_list = []
    split_list = []
    
    # split_list will generate: [('', 'cans'), ('c', 'ans'), ('ca', 'ns'), ('can', 's')]
    split_list = [(word[:i], word[i:]) for i in range(len(word))]
    
    # delete_list will generate: ['ans', 'cns', 'cas', 'can']
    # deletes one char from the word
    delete_list = [L+R[1:] for L, R in split_list]
    

    if verbose: print(f"input word {word}, \nsplit_list = {split_list}, \ndelete_list = {delete_list}")

    return delete_list

In [10]:
# Example
delete_word_list = delete_letter(word="cans",verbose=True)

input word cans, 
split_list = [('', 'cans'), ('c', 'ans'), ('ca', 'ns'), ('can', 's')], 
delete_list = ['ans', 'cns', 'cas', 'can']


### Switch Letter

In [11]:
def switch_letter(word, verbose=False):
    '''
    Input:
        word: input string
     Output:
        switches: a list of all possible strings with one adjacent charater switched
    ''' 
    
    switch_list = []
    split_list = []

    split_list = [(word[:i], word[i:]) for i in range(len(word))]
    #.  a.    b. 
    # ('e', 'ta') => L + R[1] + R[0] + R[2:] => eat
    switch_list = [L + R[1] + R[0] + R[2:] for L, R in split_list if len(R)>=2]
    
    if verbose: print(f"Input word = {word} \nsplit_list = {split_list} \nswitch_list = {switch_list}") 

    return switch_list


In [12]:
# Example
switch_word_list = switch_letter(word="eta",verbose=True)

Input word = eta 
split_list = [('', 'eta'), ('e', 'ta'), ('et', 'a')] 
switch_list = ['tea', 'eat']


### Replace letter

In [13]:
def replace_letter(word, verbose=False):
    '''
    Input:
        word: the input string/word 
    Output:
        replaces: a list of all possible strings where we replaced one letter from the original word. 
    ''' 
    
    letters = 'abcdefghijklmnopqrstuvwxyz'
    replace_list = []
    split_list = []
    
    split_list = [(word[0:i], word[i:]) for i in range(len(word))]
    
    replace_list = [L + letter + (R[1:] if len(R)>1 else '') for L, R in split_list if R for letter in letters]
    
    replace_set = set(replace_list)
    
    # turn the set back into a list and sort it, for easier viewing
    replace_list = sorted(list(replace_set))

    
    if verbose: print(f"Input word = {word} \nsplit_list = {split_list} \nreplace_list {replace_list}")   
    
    return replace_list

In [14]:
# Example
replace_list = replace_letter(word='can', verbose=True)

Input word = can 
split_list = [('', 'can'), ('c', 'an'), ('ca', 'n')] 
replace_list ['aan', 'ban', 'caa', 'cab', 'cac', 'cad', 'cae', 'caf', 'cag', 'cah', 'cai', 'caj', 'cak', 'cal', 'cam', 'can', 'cao', 'cap', 'caq', 'car', 'cas', 'cat', 'cau', 'cav', 'caw', 'cax', 'cay', 'caz', 'cbn', 'ccn', 'cdn', 'cen', 'cfn', 'cgn', 'chn', 'cin', 'cjn', 'ckn', 'cln', 'cmn', 'cnn', 'con', 'cpn', 'cqn', 'crn', 'csn', 'ctn', 'cun', 'cvn', 'cwn', 'cxn', 'cyn', 'czn', 'dan', 'ean', 'fan', 'gan', 'han', 'ian', 'jan', 'kan', 'lan', 'man', 'nan', 'oan', 'pan', 'qan', 'ran', 'san', 'tan', 'uan', 'van', 'wan', 'xan', 'yan', 'zan']


### Insert Letter

In [15]:
def insert_letter(word, verbose=False):
    '''
    Input:
        word: the input string/word 
    Output:
        inserts: a set of all possible strings with one new letter inserted at every offset
    ''' 
    letters = 'abcdefghijklmnopqrstuvwxyz'
    insert_list = []
    split_list = []
    
    # adding extra 1 because for insertion, we want that empty tuple!
    split_list = [(word[0:i], word[i:]) for i in range(len(word)+1)]
    insert_list = [L + letter + R for L, R in split_list for letter in letters]

    if verbose: print(f"Input word {word} \nsplit_list = {split_list} \ninsert_list = {insert_list}")
    
    return insert_list

In [16]:
# Example
insert_list = insert_letter('at', True)
print(f"Number of strings output by insert_letter('at') is {len(insert_list)}")

Input word at 
split_list = [('', 'at'), ('a', 't'), ('at', '')] 
insert_list = ['aat', 'bat', 'cat', 'dat', 'eat', 'fat', 'gat', 'hat', 'iat', 'jat', 'kat', 'lat', 'mat', 'nat', 'oat', 'pat', 'qat', 'rat', 'sat', 'tat', 'uat', 'vat', 'wat', 'xat', 'yat', 'zat', 'aat', 'abt', 'act', 'adt', 'aet', 'aft', 'agt', 'aht', 'ait', 'ajt', 'akt', 'alt', 'amt', 'ant', 'aot', 'apt', 'aqt', 'art', 'ast', 'att', 'aut', 'avt', 'awt', 'axt', 'ayt', 'azt', 'ata', 'atb', 'atc', 'atd', 'ate', 'atf', 'atg', 'ath', 'ati', 'atj', 'atk', 'atl', 'atm', 'atn', 'ato', 'atp', 'atq', 'atr', 'ats', 'att', 'atu', 'atv', 'atw', 'atx', 'aty', 'atz']
Number of strings output by insert_letter('at') is 78


## Combining the Edits
Now that we have implemented a number of string manipulations functions (insert, delete, switch and replace), we will now create two functions that, given a string, will return all the possible single and double edits on that string. These will be `edit_one_letter()` and `edit_two_letters()`.

### Edit One Letter

In [17]:
def edit_one_letter(word, allow_switches = True):
    """
    Input:
        word: the string/word for which we will generate all possible wordsthat are one edit away.
    Output:
        edit_one_set: a set of words with one possible edit. Returning a set to avoid duplicates.
    """
    
    edit_one_set = set()
    
    edit_one_set.update(delete_letter(word))
    if allow_switches:
        edit_one_set.update(switch_letter(word))
    edit_one_set.update(replace_letter(word))
    edit_one_set.update(insert_letter(word))
    
    if word in edit_one_set:
        edit_one_set.remove(word)


    return edit_one_set

In [18]:
# Example
tmp_word = "at"
tmp_edit_one_set = edit_one_letter(tmp_word)

# turn this into a list to sort it, in order to view it
tmp_edit_one_list = sorted(list(tmp_edit_one_set))

print(f"input word {tmp_word} \nedit_one_l \n{tmp_edit_one_list}\n")
print(f"The type of the returned object should be a set {type(tmp_edit_one_set)}")
print(f"Number of outputs from edit_one_letter('at') is {len(edit_one_letter('at'))}")

input word at 
edit_one_l 
['a', 'aa', 'aat', 'ab', 'abt', 'ac', 'act', 'ad', 'adt', 'ae', 'aet', 'af', 'aft', 'ag', 'agt', 'ah', 'aht', 'ai', 'ait', 'aj', 'ajt', 'ak', 'akt', 'al', 'alt', 'am', 'amt', 'an', 'ant', 'ao', 'aot', 'ap', 'apt', 'aq', 'aqt', 'ar', 'art', 'as', 'ast', 'ata', 'atb', 'atc', 'atd', 'ate', 'atf', 'atg', 'ath', 'ati', 'atj', 'atk', 'atl', 'atm', 'atn', 'ato', 'atp', 'atq', 'atr', 'ats', 'att', 'atu', 'atv', 'atw', 'atx', 'aty', 'atz', 'au', 'aut', 'av', 'avt', 'aw', 'awt', 'ax', 'axt', 'ay', 'ayt', 'az', 'azt', 'bat', 'bt', 'cat', 'ct', 'dat', 'dt', 'eat', 'et', 'fat', 'ft', 'gat', 'gt', 'hat', 'ht', 'iat', 'it', 'jat', 'jt', 'kat', 'kt', 'lat', 'lt', 'mat', 'mt', 'nat', 'nt', 'oat', 'ot', 'pat', 'pt', 'qat', 'qt', 'rat', 'rt', 'sat', 'st', 't', 'ta', 'tat', 'tt', 'uat', 'ut', 'vat', 'vt', 'wat', 'wt', 'xat', 'xt', 'yat', 'yt', 'zat', 'zt']

The type of the returned object should be a set <class 'set'>
Number of outputs from edit_one_letter('at') is 129


### Edit two letters

In [19]:
def edit_two_letters(word, allow_switches = True):
    '''
    Input:
        word: the input string/word 
    Output:
        edit_two_set: a set of strings with all possible two edits
    '''
    
    edit_two_set = set()
    edit_one = edit_one_letter(word, allow_switches=allow_switches)
    for word in edit_one:
        if word:
            edit_two = edit_one_letter(word, allow_switches=allow_switches)
            edit_two_set.update(edit_two)
    
    return edit_two_set


In [20]:
insert_list_with_2_edits = []
insertion_list_with_1_edit = insert_letter("a")
for word in insertion_list_with_1_edit:
    insert_list_with_2_edits += insert_letter(word)

print(len(set(insert_list_with_2_edits)))


1951


In [21]:
# Example
tmp_edit_two_set = edit_two_letters("a")
tmp_edit_two_list = sorted(list(tmp_edit_two_set))
print(f"Number of strings with edit distance of two: {len(tmp_edit_two_list)}")
print(f"First 10 strings {tmp_edit_two_list[:10]}")
print(f"Last 10 strings {tmp_edit_two_list[-10:]}")
print(f"The data type of the returned object should be a set {type(tmp_edit_two_set)}")
print(f"Number of strings that are 2 edit distances from 'at' is {len(edit_two_letters('at'))}")

Number of strings with edit distance of two: 2654
First 10 strings ['', 'a', 'aa', 'aaa', 'aab', 'aac', 'aad', 'aae', 'aaf', 'aag']
Last 10 strings ['zv', 'zva', 'zw', 'zwa', 'zx', 'zxa', 'zy', 'zya', 'zz', 'zza']
The data type of the returned object should be a set <class 'set'>
Number of strings that are 2 edit distances from 'at' is 7154


## Spelling Suggestions

Now we will use our edit_two_letters function to get a set of all the possible 2 edits on our word. We will then use those strings to get the most probable word that we meant to type aka our typing suggestion. For generating suggestions for a supplied word: We'll use the edit functions we have developed. The 'suggestion algorithm' follows the below logic:

1. If the word is in the vocabulary, suggest the word.
2. Otherwise, if there are suggestions from edit_one_letter that are in the vocabulary, use those.
3. Otherwise, if there are suggestions from edit_two_letters that are in the vocabulary, use those.
4. Otherwise, suggest the input word.

The idea is that words generated from fewer edits are more likely than words with more edits.

In [22]:
def get_spelling_suggestions(word, probs, vocab, n=2, verbose = False):
    '''
    Input: 
        word: a user entered string to check for suggestions
        probs: a dictionary that maps each word to its probability in the corpus
        vocab: a set containing all the vocabulary
        n: number of possible word corrections you want returned in the dictionary
    Output: 
        n_best: a list of tuples with the most probable n corrected words and their probabilities.
    '''
    
    suggestions = []
    top_n_suggestions = []
    
    # [] or [] or ["least","of","all"] # continues evaluation until there is a non-empty list
    suggestions = list((word in vocab and word) or edit_one_letter(word).intersection(vocab) or edit_two_letter(word).intersection(vocab))
    top_n_suggestions = [[s, probs[s]] for s in list(suggestions)]
    
    if verbose: print("entered word = ", word, "\nsuggestions = ", suggestions)

    return top_n_suggestions


#### Outputs:

Edit `my_words` according to you. If it gives error it means word is not present in our dataset

In [23]:
my_words = ['dys', 'nghts', 'aple', 'mt', 'cre']
for _ in my_words: 

  tmp_corrections = get_spelling_suggestions(_, probs, vocab, 3, verbose=True) # keep verbose=True
  for i, word_prob in enumerate(tmp_corrections):
      print(f"word {i}: {word_prob[0]}, probability {word_prob[1]:.6f}")
  print("------------------")      

entered word =  dys 
suggestions =  ['days', 'dye', 'dy']
word 0: days, probability 0.000976
word 1: dye, probability 0.000046
word 2: dy, probability 0.000093
------------------
entered word =  nghts 
suggestions =  ['nights']
word 0: nights, probability 0.000093
------------------
entered word =  aple 
suggestions =  ['able', 'apple', 'pale']
word 0: able, probability 0.000046
word 1: apple, probability 0.000046
word 2: pale, probability 0.000093
------------------
entered word =  mt 
suggestions =  ['tm', 'at', 'my', 't', 'st', 'me', 'it', 'met', 'ut']
word 0: tm, probability 0.002649
word 1: at, probability 0.001859
word 2: my, probability 0.017288
word 3: t, probability 0.000186
word 4: st, probability 0.002510
word 5: me, probability 0.007622
word 6: it, probability 0.005530
word 7: met, probability 0.000046
word 8: ut, probability 0.000046
------------------
entered word =  cre 
suggestions =  ['cure', 'care', 'cry', 're', 'ere', 'are']
word 0: cure, probability 0.000232
word 1:

## Minimum Edit Distance
Now, how do we efficiently find the shortest path to go from the word, 'waht' to the word 'what'? In this section, we will implement a dynamic programming system that will tell us the minimum number of edits required to convert a string into another string.

### Dynamic Programming

In [24]:
def min_edit_distance(source, target, ins_cost = 1, del_cost = 1, rep_cost = 2):
    '''
    Input: 
        source: a string corresponding to the string we are starting with
        target: a string corresponding to the string we want to end with
        ins_cost: an integer setting the insert cost
        del_cost: an integer setting the delete cost
        rep_cost: an integer setting the replace cost
    Output:
        D: a matrix of len(source)+1 by len(target)+1 containing minimum edit distances
        med: the minimum edit distance (med) required to convert the source string to the target
    '''
    # use deletion and insert cost as  1
    m = len(source) 
    n = len(target) 
    
    #initialize cost matrix with zeros and dimensions (m+1,n+1) 
    D = np.zeros((m+1, n+1), dtype=int) 
    
    # Fill in column 0, from row 1 to row m, both inclusive
    for row in range(1,m+1): 
        D[row,0] = D[row-1, 0]+del_cost
        
    # Fill in row 0, for all columns from 1 to n, both inclusive
    for col in range(1,n+1): 
        D[0,col] = D[0, col-1]+ins_cost
        
    # Loop through row 1 to row m, both inclusive
    for row in range(1,m+1): 
        
        # Loop through column 1 to column n, both inclusive
        for col in range(1,n+1):
            
            # Intialize r_cost to the 'replace' cost that is passed into this function
            r_cost = rep_cost
            
            # Check to see if source character at the previous row
            # matches the target character at the previous column, 
            # (source) 'TAY' (target) 'SAY': Check if 'T'=='S'
            if source[row-1] == target[col-1]:
                # Update the replacement cost to 0 if source and target are the same
                r_cost = 0
                
            # Update the cost at row, col based on previous entries in the cost matrix
            # Refer to the equation calculate for D[i,j] (the minimum of three calculated costs)

            cost_del = D[row-1, col] + del_cost
            cost_ins = D[row, col-1] + ins_cost
            cost_replace = D[row-1, col-1] + r_cost
            
            D[row,col] = min(cost_del, cost_ins, cost_replace)
          
    # Set the minimum edit distance with the cost found at row m, column n
    med = D[m, n]
    
    return D, med


In [25]:
# Example
source =  'eer'
target = 'near'
matrix, min_edits = min_edit_distance(source, target)
print("minimum edits: ",min_edits, "\n")
idx = list('#' + source)
cols = list('#' + target)
df = pd.DataFrame(matrix, index=idx, columns= cols)
print(df)

minimum edits:  3 

   #  n  e  a  r
#  0  1  2  3  4
e  1  2  1  2  3
e  2  3  2  3  4
r  3  4  3  4  3
